# ETL GOOGLE

# 1. Importar librerias

In [1]:
import pandas as pd
import numpy as np
import os
import requests

# 2. Carga de Datos

In [2]:
dfBusinessGoogle = pd.read_csv('DataSets/businessGoogle.csv')

In [3]:
dfReviewGoogle = pd.read_csv('DataSets/reviewGoogle.csv')

# 3. Exploracion y Limpieza de Datos

## 3.1 Dataset Business Yelp

### 3.1.1. Exploración inicial de los datos

In [4]:
dfBusinessGoogle.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,NaN,32.388300,-83.357100,['Pharmacy'],4.9,16,NaN,"[['Friday', '8AM–6PM'], ['Saturday', '8AM–12PM...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"['0x88f16e41929435cf:0x5b2532a2885e9ef6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
1,MemorialCare Breast Center - Irvine Sand Canyon,MemorialCare Breast Center - Irvine Sand Canyo...,0x80dcdd3b95c7284b:0x23d9b4440c982d37,NaN,33.668389,-117.765649,['Mammography service'],4.7,33,NaN,"[['Tuesday', '8:30AM–5PM'], ['Wednesday', '8:3...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8:30AM,"['0x80dcdc4aad8cb91f:0x62bdeb79a44d7633', '0x8...",https://www.google.com/maps/place//data=!4m2!3...
2,U-Haul Neighborhood Dealer,"U-Haul Neighborhood Dealer, 1690 E 112th Ave, ...",0x876c769e600eda1d:0xdf1fffc2e00e1fe1,NaN,39.899189,-104.966765,"['Truck rental agency', 'Trailer rental servic...",3.9,18,$,NaN,NaN,NaN,"['0x876c75d05d0e1bf3:0x1d2d9897d0dec9cb', '0x8...",https://www.google.com/maps/place//data=!4m2!3...


In [5]:
dfBusinessGoogle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274996 non-null  object 
 1   address           269428 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       27900 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          273662 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             31139 non-null   object 
 10  hours             210287 non-null  object 
 11  MISC              221703 non-null  object 
 12  state             212960 non-null  object 
 13  relative_results  254406 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [6]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfBusinessGoogle.isnull().sum()
print(filas_vacias_por_columna)

name                     5
address               5573
gmap_id                  0
description         247101
latitude                 0
longitude                0
category              1339
avg_rating               0
num_of_reviews           0
price               243862
hours                64714
MISC                 53298
state                62041
relative_results     20595
url                      0
dtype: int64


In [7]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfBusinessGoogle.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 0


### 3.1.2. Limpieza y Normalización de datos

#### Filtrado de datos

In [8]:
# El alcance del proyecto se limita a los diferentes negocios 
# dentro de la categoria 'Nail Salon', por lo que se aplica el filtro.

# Eliminar filas con valores NaN en la columna 'categories'
dfBusinessGoogle = dfBusinessGoogle.dropna(subset=['category'])

# Filtrar las filas que contienen 'Nail Salons' en la columna 'categories'
dfBusinessGoogle = dfBusinessGoogle[dfBusinessGoogle['category'].str.contains('Nail salon', na=False)]

De limitamos a una franquicia de comida rapida 'Subway'(sandwiches) que hemos puesto como objetivo y delimitador

In [9]:
dfBusinessGoogle = dfBusinessGoogle[dfBusinessGoogle['state'] != 'Permanently closed']
dfBusinessGoogle = dfBusinessGoogle.drop('state', axis=1) 

Realizamos un filtro donde solo nos quedamos con los establecimientos que esten activos o en funcionamiento

In [10]:
dfBusinessGoogle['city'] = dfBusinessGoogle['address'].str.split(',').str[2]

Reemplzamos la Subway por espacio y mediante la funcion split obtenemos los nombres de las ciudades y guardamos en la columna 'city'

In [11]:
dfBusinessGoogle['state_PostalCode'] = dfBusinessGoogle['address'].str.split(',').str[3]
dfBusinessGoogle['postal_code']=dfBusinessGoogle['state_PostalCode'].str.split(' ').str[2]

Mediante la función Split podemos extraer de la columna address los valores de codigo postal que guardamos en la columna 'postal_code'

In [12]:
def reverse_geocode(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
    response = requests.get(url)
    data = response.json()
    if 'address' in data:
        state = data['address'].get('state', '')
        return state

# Supongamos que dfBusinessGoogle es tu DataFrame y que 'latitude' y 'longitude' son tus columnas
dfBusinessGoogle['state'] = dfBusinessGoogle.apply(lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1)

Usando la API OpenStreetMap (OSM) se realizo una geocodificacion inversa que me devuelve los diferentes estados('state') basado en las columnas 'latitude' y 'longitude' 

In [13]:
dfBusinessGoogle = dfBusinessGoogle.drop('state_PostalCode', axis=1)

dfBusinessGoogle = dfBusinessGoogle.drop('description', axis=1)

dfBusinessGoogle = dfBusinessGoogle.drop('category', axis=1)

dfBusinessGoogle = dfBusinessGoogle.drop('price', axis=1) 

dfBusinessGoogle = dfBusinessGoogle.drop('relative_results', axis=1)

dfBusinessGoogle = dfBusinessGoogle.drop('url', axis=1)

dfBusinessGoogle = dfBusinessGoogle.drop('MISC', axis=1)


In [14]:
dfBusinessGoogle = dfBusinessGoogle.rename(columns={'gmap_id': 'business_id', 'avg_rating': 'stars', 'num_of_reviews': 'review_count'})

Renombramos las columnnas para tener los mismos nombres de la tabla Yelp(normalizar) para realizar un mejor analisis de datos

In [15]:
# Se creó la columna 'source' con un valor unico 'G' 
dfBusinessGoogle['source']='G'

Se creo la colummna 'source' para identificar la tabla que pertenecen a los datos obtenidos de las tablas Google

In [16]:
nuevo_orden_columnas = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'hours', 'source']

dfBusinessGoogle = dfBusinessGoogle[nuevo_orden_columnas]

In [17]:
dfBusinessGoogle.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,hours,source
42,0x80dcddb51e362a2f:0xea0942477ad212a6,City nails and spa,"City nails and spa, 4250 Barranca Pkwy n1, Irv...",Irvine,California,92604,33.681278,-117.804961,4.2,48,"[['Tuesday', '9:30AM–7PM'], ['Wednesday', '9:3...",G
194,0x8830f12316ab98f5:0xaeb4263777ba0bb,Gaia Nail Lounge & Spa,"Gaia Nail Lounge & Spa, 1870 W 25th St, Clevel...",Cleveland,Ohio,44113,41.485645,-81.704981,4.3,58,"[['Tuesday', '11AM–8PM'], ['Wednesday', '11AM–...",G
205,0x8830fbcda1e07e15:0xf8eb2508d7fd545c,Styles of Success | Hair Salon,"Styles of Success | Hair Salon, 6524 St Clair ...",Cleveland,Ohio,44103,41.525441,-81.644494,4.6,147,"[['Tuesday', '10AM–6PM'], ['Wednesday', '10AM–...",G
211,0x8830fdbd99eef30f:0xb69e407e123502d9,Square One Nails,"Square One Nails, 13915 Cedar Rd, South Euclid...",South Euclid,Ohio,44118,41.502357,-81.539290,2.9,118,"[['Tuesday', '10AM–7PM'], ['Wednesday', '10AM–...",G
218,0x8830fba741e5d0b1:0x626c4c7ba05e498f,Studio S,"Studio S, 850 Euclid Ave #320, Cleveland, OH 4...",Cleveland,Ohio,44114,41.499814,-81.687098,4.6,36,"[['Tuesday', 'Closed'], ['Wednesday', 'Closed'...",G


In [18]:
dfBusinessGoogle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4350 entries, 42 to 274928
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   4350 non-null   object 
 1   name          4350 non-null   object 
 2   address       4346 non-null   object 
 3   city          4346 non-null   object 
 4   state         4350 non-null   object 
 5   postal_code   4214 non-null   object 
 6   latitude      4350 non-null   float64
 7   longitude     4350 non-null   float64
 8   stars         4350 non-null   float64
 9   review_count  4350 non-null   int64  
 10  hours         4073 non-null   object 
 11  source        4350 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 441.8+ KB


## 3.2 Dataset Reviews Google

### 3.2.1. Exploración inicial de los datos

In [19]:
#Vizualizo las primeras filas del DataFrame
dfReviewGoogle.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.183560e+20,Vicki Ullery,1628718071358,5,Gwen was so informative and made me feel comfo...,NaN,NaN,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
1,1.126523e+20,Doc Kovacs,1609796129431,5,Dr. Huyhn is an ethical skilled dentist. Her s...,NaN,NaN,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
2,1.089879e+20,Pedro Diaz,1594225668169,1,Not sure if one star is fair but my situation ...,NaN,NaN,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
3,1.069277e+20,Brandon Ray,1622676189509,5,Best dentist office I've ever had. Love everyt...,NaN,NaN,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
4,1.090535e+20,T. Bennett,1434829263041,5,I've been coming to Oasis dentistry for a whil...,NaN,NaN,0x872b5dae3c5fcfa7:0xf0f8b286a2537821


In [20]:
dfReviewGoogle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10792020 entries, 0 to 10792019
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 658.7+ MB


In [21]:
# Contar filas vacías por columna
filas_vacias_por_columna = dfReviewGoogle.isnull().sum()
print(filas_vacias_por_columna)

user_id           0
name            115
time              0
rating            0
text        4429627
pics       10409585
resp        9488684
gmap_id           0
dtype: int64


Observamos que tenemos varias columnas con muchos valores nulos, vamos observando con que columnas podemos trabajar

In [22]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfReviewGoogle.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 199018


In [41]:
# Identificar las filas duplicadas en el DataFrame dfReviewGoogle
filas_duplicadas = dfReviewGoogle[dfReviewGoogle.duplicated()]

# Mostrar las filas duplicadas
print("Filas duplicadas:")
print(filas_duplicadas)

Filas duplicadas:
               user_id                name  \
1539373   1.030571e+20               Jus C   
1539378   1.067648e+20         sportsdocaz   
1539390   1.140847e+20    Victoria Baumann   
1539453   1.152130e+20         AJ Goodroad   
1539560   1.165358e+20        Zother Onez2   
...                ...                 ...   
10373676  1.158848e+20       A.D. Ferguson   
10378671  1.021558e+20        Simon Mujica   
10378676  1.177352e+20        Aisha Walker   
10381067  1.094389e+20  Brenda9192 Domingo   
10381070  1.021314e+20       Amberly Rhodd   

                                    business_id  review_stars  \
1539373   0x872b626d7dc99bf9:0xf7ea9221b5343866           5.0   
1539378   0x872b626d7dc99bf9:0xf7ea9221b5343866           1.0   
1539390   0x872b626d7dc99bf9:0xf7ea9221b5343866           5.0   
1539453   0x872b634486bf519f:0xeb4cca161bd506cd           5.0   
1539560   0x86d5e1b2313ba265:0xb6a0838b5cb9a057           5.0   
...                                    

Observamos muchas filas duplicadas, pero tenemos que corrobarar con el filtro de codigo de bussines y continuaremos

### 3.2.2. Limpieza y Normalización de datos

Filtrado de datos

In [23]:
# Obtener los id únicos del DataFrame dfbusinessYelp
business_ids_to_keep = dfBusinessGoogle['business_id'].unique()

# Friltamos el dfreviewYelp basado en los balores unico de lao id de dfbusinessYelp
dfReviewGoogle = dfReviewGoogle[dfReviewGoogle['gmap_id'].isin(business_ids_to_keep)]


Aplicamos un filtro solo para quedarnos con los valores que coincidan con el id del negocio

In [24]:
# Convertir la columna 'time' de int64 a datetime
dfReviewGoogle['time'] = pd.to_datetime(dfReviewGoogle['time'], unit='ms')

La columna 'time' tiene un forma o tipo de dato int y lo transforme en DateTime

In [25]:
columnas_a_eliminar = ['pics', 'resp']

# Eliminar las columnas especificadas
dfReviewGoogle = dfReviewGoogle.drop(columnas_a_eliminar, axis=1)

In [26]:
# Aplicar la limpieza de texto directamente sobre la columna 'text'
dfReviewGoogle['text'] = dfReviewGoogle['text'].str.lower().str.replace(r'[^a-z0-9\s]', '')

In [27]:
# Modificamos el tipo de dato a un valor flotante
dfReviewGoogle['rating'] = dfReviewGoogle['rating'].astype('float64')

In [28]:
dfReviewGoogle = dfReviewGoogle.rename(columns={'gmap_id': 'business_id', 'rating': 'review_stars'})

In [34]:
dfReviewGoogle = dfReviewGoogle.rename(columns={'time': 'date'})

In [36]:
nuevo_orden_columnas = ['user_id','name', 'business_id', 'review_stars', 'text', 'date']
dfReviewGoogle = dfReviewGoogle[nuevo_orden_columnas]

In [ ]:
dfReviewGoogle['source']='G'

In [ ]:
dfReviewGoogle.head()

,user_id,business_id,stars,text,year,month,day,source
1538921,1.062874e+20,0x872b1129edb01139:0xdf5a1677919571fd,1.0,one star is too much for this place\n2 months ...,2020,9,18,G
1538922,1.112756e+20,0x872b1129edb01139:0xdf5a1677919571fd,1.0,the girls were patient and friendly but how do...,2020,11,23,G
1538923,1.083110e+20,0x872b1129edb01139:0xdf5a1677919571fd,1.0,every thing looks bad pickles spinach tun...,2021,6,2,G
1538924,1.006935e+20,0x872b1129edb01139:0xdf5a1677919571fd,1.0,dirty subway. don't waste your money here. i w...,2020,1,31,G
1538925,1.126158e+20,0x872b1129edb01139:0xdf5a1677919571fd,2.0,i ordered 5 ft longs online. 10 minus after my...,2020,8,8,G


## 4. Union y Almacenamiento de Datos

In [37]:
dfReviewGoogle.to_csv('DataSets/Cleaned_Review_G.csv', index = False)


In [30]:
dfBusinessGoogle.to_csv('DataSets/Cleaned_Business_G.csv', index=False)

In [38]:
# Contar la cantidad de duplicados del DataFrame
filas_duplicadas = dfReviewGoogle.duplicated().sum()
print("Filas duplicadas en el DataFrame:", filas_duplicadas)

Filas duplicadas en el DataFrame: 174


In [40]:
dfReviewGoogle.shape

(9435, 6)

In [39]:
# Eliminar filas duplicadas en el DataFrame dfReviewGoogle
dfReviewGoogle_sin_duplicados = dfReviewGoogle.drop_duplicates()

# Verificar el nuevo tamaño del DataFrame después de eliminar duplicados
print("Tamaño del DataFrame después de eliminar duplicados:", dfReviewGoogle_sin_duplicados.shape)

Tamaño del DataFrame después de eliminar duplicados: (9261, 6)
